# Rule based bot - Marauding wingers

Just a simple rule based bot that I was able to put together. I started with the sample submission, then tried to build out a rule based game strategy. I drew from my experience playing FIFA and this strategy is inspired by those I've played against who just run down the wing mindlessly and put in crosses. I find the cross-heavy gameplay particularly annoying. Not because it is particularly effective, but it just leads to a lot of throw ins and stoppages in play since the ball is concentrated close to the byline. Not fun to have a lot of stoppages and interruptions when you're playing FIFA
Anyway, back to the bot. It is built to run down the wing and bang in crosses. One thing I haven't been able to figure out is how to code in first-time shots. I started a discussion on that [here](https://www.kaggle.com/c/google-football/discussion/196035). 

This bot does pretty well for being so simple. It reached a score of 1105 at one point, and now **stands at 1041.1 as of 11/09.** I wanted to add this write up so you might see the score different on this notebook when I put this out there.

There are still many other rule-based strategies that I had in mind. Like not to be sprinting all the time like this bot does, or have a better approach at finding open teammates for passing etc. But I want to move on to building out some algorithmic solutions and learn some new techniques. I don't have much experience with Reinforcement Learning and my aim is to build an RL solution that beats this benchmark I have set for myself before the competitions ends. 

Leave me any feedback/ questions in the comments and feel free to share some good resources! 

Here are some useful resources for ideas on the rule-based approach. If you see your ideas on this notebook and I haven't mentioned your notebook, let me know!

1. https://github.com/google-research/football/blob/master/gfootball/doc/observation.md#raw-observations
2. https://github.com/Kaggle/kaggle-environments/blob/28e75806001c51f1fecbd5d3d8fe7b0aa83f33e5/kaggle_environments/envs/football/helpers.py
3. https://www.kaggle.com/c/google-football/discussion/191804
4. https://www.kaggle.com/piotrstanczyk/gfootball-template-bot
5. https://www.kaggle.com/mlconsult/best-open-rules-bot-score-1020-7

In [ ]:
%%capture
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.8 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.8.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *
from random import randint


# Function to calculate distance 
def get_distance(pos1,pos2):
    return (((pos1[0]-pos2[0])**2)+((pos1[1]-pos2[1])**2))**0.5

# Function to cross ball from wing
def cross_ball():
    pass

# Movement directions
directions = [
[Action.TopLeft, Action.Top, Action.TopRight],
[Action.Left, Action.Idle, Action.Right],
[Action.BottomLeft, Action.Bottom, Action.BottomRight]]

dirsign = lambda x: 1 if abs(x) < 0.01 else (0 if x < 0 else 2)

# Set game plan parameters
goalRange = 0.65
wingRange = 0.21

@human_readable_agent
def agent(obs):
    
    # Add direction to action
    def sticky_check(action, direction):
        if direction in obs['sticky_actions']:
            return action
        else:
            return direction
    
    controlled_player_pos = obs['left_team'][obs['active']]
    
    
    # Pass when KickOff or ThrowIn
    if obs['game_mode'] == GameMode.KickOff or obs['game_mode'] == GameMode.ThrowIn:
        return sticky_check(Action.ShortPass, Action.Right) 
    
    # Shoot when freekick in goal range; If on wing then cross; Otherwise just pass
    if obs['game_mode'] == GameMode.FreeKick:
        # Shoot if in range
        if controlled_player_pos[0] > goalRange and controlled_player_pos[1] < wingRange and controlled_player_pos[1] > -(wingRange):
            ydir = randint(0,2)
            return sticky_check(Action.Shot, directions[ydir][2]) 
        # Cross from right
        if controlled_player_pos[0] > goalRange and controlled_player_pos[1] > wingRange:
            return sticky_check(Action.HighPass, Action.TopRight)
        
        # Cross from left
        if controlled_player_pos[0] > goalRange and controlled_player_pos[1] < -(wingRange):
            return sticky_check(Action.HighPass, Action.BottomRight)
    
    # Cross in for corner
    if obs['game_mode'] == GameMode.Corner and obs['ball'][1] < 0:
        return sticky_check(Action.HighPass, Action.Bottom)
    elif obs['game_mode'] == GameMode.Corner and obs['ball'][1] > 0:
        return sticky_check(Action.HighPass, Action.Top)
        
    # High pass when GoalKick 
    if obs['game_mode'] == GameMode.GoalKick:
        ydir = randint(0,2)
        return sticky_check(Action.HighPass, directions[ydir][2])
    
    # Shoot when Penalty
    if obs['game_mode'] == GameMode.Penalty:
        xdir = randint(0,2)
        ydir = randint(0,2)
        return sticky_check(Action.Shot, directions[ydir][xdir])
    
    # Make sure player is running.
    if Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    # We always control left team (observations and actions
    # are mirrored appropriately by the environment).
    controlled_player_pos = obs['left_team'][obs['active']]
    
    # Check if we are in possession
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        
        # Clear if we are near our goal
        if controlled_player_pos[0] < -(goalRange):
            return sticky_check(Action.HighPass, Action.Right)
        
        # Shoot if we are in the final third and not at an acute angle
        if controlled_player_pos[0] > goalRange and controlled_player_pos[1] < wingRange and controlled_player_pos[1] > -(wingRange):
            ydir = randint(0,2)
            return sticky_check(Action.Shot, directions[ydir][2])
        #if the goalie is coming out on player near goal shoot
        elif obs['right_team'][0][0] < 0.8 or abs(obs['right_team'][0][1]) > 0.05:
            return Action.Shot
        
        # Cross from right
        if controlled_player_pos[0] > goalRange and controlled_player_pos[1] > wingRange:
            return sticky_check(Action.HighPass, Action.TopRight)
        
        # Cross from left
        if controlled_player_pos[0] > goalRange and controlled_player_pos[1] < -(wingRange):
            return sticky_check(Action.HighPass, Action.BottomRight)
        
        # Run towards the goal otherwise.
        return Action.Right
    else:
        #where ball is going we add the direction xy to ball current location
        ball_targetx=obs['ball'][0]+(1.5 * obs['ball_direction'][0])
        ball_targety=obs['ball'][1]+(1.5 * obs['ball_direction'][1])

        # Euclidian distance to ball
        e_dist=get_distance(obs['left_team'][obs['active']],obs['ball'])

        if e_dist >.005:
            # Run where ball will be
            xdir = dirsign(ball_targetx - controlled_player_pos[0])
            ydir = dirsign(ball_targety - controlled_player_pos[1])
            return directions[ydir][xdir]
        else:
            prob = randint(0,100)
            if prob > 70 and controlled_player_pos[0] < obs['right_team'][obs['active']][0]:
                return Action.Slide
            # Run towards the ball.
            xdir = dirsign(obs['ball'][0] - controlled_player_pos[0])
            ydir = dirsign(obs['ball'][1] - controlled_player_pos[1])
            return directions[ydir][xdir]


In [ ]:
# Set up the Environment.
from kaggle_environments import make
env = make("football", debug=True, configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle/working/submission.py", "do_nothing"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)

In [ ]:
# Dont slide from behind
# Find player density on pitch to pass to open space / run into open space
# Dribbling
# Shoot if goalie off the line